# **HMM PROJECT Using Log**

## Why We Use Logarithms in HMM (Log-Space)

In DNA HMM models, the values of α (forward), β (backward), Gamma, and Xi
become extremely small because each step multiplies many probabilities
(less than 1).

This causes **numerical underflow**, where values approach zero and the
computer rounds them to 0.0, which breaks the Baum–Welch updates.

To fix this, we perform all HMM calculations in **log-space**:

- multiplication becomes addition  
- division becomes subtraction  
- sums use the log-sum-exp trick  
- no value ever becomes zero  
- results stay stable and meaningful  

This allows the model to learn correct parameters even for long DNA sequences.


## 1. Dataset Prepration

### DNA sequence dataset

https://www.kaggle.com/datasets/nageshsingh/dna-sequence-dataset?resource=download&select=chimpanzee.txt

**Data description for HMM**

The dataset contains sequences of DNA nucleotides: A (Adenine), C (Cytosine), G (Guanine), T (Thymine).

These letters are the observations that will be used by the Hidden Markov Model (HMM).

Each sequence is treated as a separate observation sequence.

The hidden states (e.g., active or inactive gene regions) are unknown and will be estimated by the HMM using the Baum–Welch algorithm.

For HMM implementation, the DNA letters are converted into numerical values:

A → 0, C → 1, G → 2, T → 3


This allows the model to learn:

Transition probabilities between hidden states.

Emission probabilities from each hidden state to the observed nucleotides.

Initial state probabilities.


---



**File description**

**human.txt** – Contains DNA sequences of the human genome. Each line represents a separate DNA sequence.



---



### CODING


In [ ]:
import glob


1. Load TXT Sequences

In [ ]:
txt_sequences = []
for filename in glob.glob("*.txt"):
    with open(filename, "r") as f:
        next(f)
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) > 0:
                seq = parts[0].upper()
                if seq != "":
                    txt_sequences.append(seq)

print("TXT sequences loaded:", len(txt_sequences))


TXT sequences loaded: 4380


## 2. Encoding Observation

Encoding (A,C,G,T → 0,1,2,3)

In [ ]:
mapping = {"A": 0, "C": 1, "G": 2, "T": 3}

observations = []
for seq in txt_sequences:
    numeric_seq = [mapping[ch] for ch in seq if ch in mapping]
    observations.append(numeric_seq)

print("Example encoded sequence:", observations[0][:50])

Example encoded sequence: [0, 3, 2, 1, 1, 1, 1, 0, 0, 1, 3, 0, 0, 0, 3, 0, 1, 3, 0, 1, 1, 2, 3, 0, 3, 2, 2, 1, 1, 1, 0, 1, 1, 0, 3, 0, 0, 3, 3, 0, 1, 1, 1, 1, 1, 0, 3, 0, 1, 3]


number of sequences

In [ ]:
num_sequences = len(observations)
print("Number of sequences:", num_sequences)


Number of sequences: 4380


sequence lengths

In [ ]:
sequence_lengths = [len(seq) for seq in observations]
print("Lengths of sequences:", sequence_lengths[:20])


Lengths of sequences: [207, 681, 1686, 1206, 1437, 1686, 1623, 1437, 1623, 644, 487, 356, 189, 98, 85, 1686, 2616, 2616, 1659, 2295]


observations = [
  
    [0,2,3,1,1,2,3,0,...],   # sequence 1 after encoding
    [1,1,0,2,3,0,1,...],     # sequence 2
    [2,2,0,1,3,3,0,...],     # sequence 3
    ...
]

## 3. Initialize HMM Parameters (A, B, π)

In [ ]:
import numpy as np

N = 2  # Number of hidden states
M = 4  # Number of observation symbols (A, C, G, T)

# Initialize Initial State Probabilities (pi)
# Create N random positive values
pi_random = np.random.rand(N)
# Normalization so that the sum = 1
initial_pi = pi_random / np.sum(pi_random)

# Initialize Transition Matrix (A)
# Create an N x N matrix of random values
A_random = np.random.rand(N, N)
# Normalize each row individually so that the sum of each row = 1
initial_A = A_random / np.sum(A_random, axis=1, keepdims=True)

# Initialize Emission Matrix (B)
# Create an N x M matrix of random values
B_random = np.random.rand(N, M)
# Normalize each row individually so that the sum of each row = 1
initial_B = B_random / np.sum(B_random, axis=1, keepdims=True)

print("Initial State Probabilities (pi):\n", initial_pi)
print("\n Transition Matrix (A):\n", initial_A)
print("\n Emission Matrix (B):\n", initial_B)

# Verification checks (for confirmation):
print("\nVerification - sum of pi:", np.sum(initial_pi))
print("Verification - sum of rows in A:", np.sum(initial_A, axis=1))
print("Verification - sum of rows in B:", np.sum(initial_B, axis=1))

Initial State Probabilities (pi):
 [0.893393 0.106607]

 Transition Matrix (A):
 [[0.22362841 0.77637159]
 [0.57803017 0.42196983]]

 Emission Matrix (B):
 [[0.14012707 0.45311353 0.34302472 0.06373468]
 [0.081004   0.22437841 0.10764351 0.58697409]]

Verification - sum of pi: 1.0
Verification - sum of rows in A: [1. 1.]
Verification - sum of rows in B: [1. 1.]


## 4. Forward Algorithm

In [ ]:
import pandas as pd
# forward  to calculate the total prob of obs start from left to right
def forword(seq,pi,Transition_matrix ,Emition_matrix):
  # we create alpha matrix to put all value of alpha , 2--> gen is active or not active
    alphamatrix=np.zeros((len(seq),2))
    # make seq int to can arrive to has emotion prob in B matrix
    seq=list(map(int,seq))
    # alpha0 for active and noactive , save in matrix
    alphanoactive=pi[0]+Emition_matrix[0][seq[0]]
    alphaactive=pi[1]+Emition_matrix[1][seq[0]]
    alphamatrix[0,0]=alphanoactive
    alphamatrix[0,1]=alphaactive
    # loop to calculate all alpha
    for i in range(1,len(seq)):
       prev_alphanoactive=alphanoactive
       prev_alphaactive=alphaactive
       #  i --> alpha 1,2,3  j-->state  . alpha[i, j] = np.sum(alpha[i-1,:] * A[:, j]) * B[j, obs[i]] general form but I use simple clear way instead , save all result
       alphanoactive=np.logaddexp.reduce(alphamatrix[i-1,:]+Transition_matrix[:,0]) + Emition_matrix[0][seq[i]]
       alphaactive=np.logaddexp.reduce(alphamatrix[i-1,:]+Transition_matrix[:,1] ) + Emition_matrix[1][seq[i]]
       alphamatrix[i,0]=alphanoactive
       alphamatrix[i,1]=alphaactive
       # create dataframe to make the result more visual
    df = pd.DataFrame({
           "seq": [i for i in seq],
           "alpha_noactive": alphamatrix[:,0],
           "alpha_active": alphamatrix[:,1],
           'alpha':[i+1 for i in range (len(seq))]  })
    df = df.set_index('alpha')
    # return the matrix to use in model and data frame to see for understanding
    return alphamatrix ,df
    # call the function
alphamatrix,df1=forword(observations[0],np.log(initial_pi),np.log(initial_A),np.log(initial_B))
print(alphamatrix[10])

[-18.34021065 -15.62359738]


In [ ]:
df1

,seq,alpha_noactive,alpha_active
alpha,,,
1,0,-2.077934,-4.751863
2,3,-6.164658,-2.827027
3,2,-4.431460,-5.855471
4,1,-6.237012,-6.056036
5,1,-7.116001,-7.482969
...,...,...,...
203,1,-318.458113,-319.382156
204,1,-320.041467,-320.010316
205,3,-322.993007,-320.382223


## 5. Backward Algorithm

In [ ]:
def backward_log(obs_seq, logA, logB, log_pi):
    """
    Backward algorithm in LOG-SPACE.
    Returns:
        log_likelihood , log_beta
    """

    T = len(obs_seq)
    N = logA.shape[0]

    log_beta = np.zeros((T, N))

    # ---- Initialization: beta[T-1,i] = log(1) = 0 ----
    log_beta[T-1, :] = 0

    # ---- Recursion ----
    for t in range(T-2, -1, -1):
        for i in range(N):

            terms = logA[i, :] + logB[:, obs_seq[t+1]] + log_beta[t+1, :]
            # log_beta[t,i] = logsumexp(terms)
            max_val = np.max(terms)
            log_beta[t, i] = max_val + np.log(np.sum(np.exp(terms - max_val)))

    # ---- Total log-likelihood ----
    first_terms = log_pi + logB[:, obs_seq[0]] + log_beta[0, :]
    max_val = np.max(first_terms)
    log_likelihood = max_val + np.log(np.sum(np.exp(first_terms - max_val)))

    return log_likelihood, log_beta


### Testing the Code

In [ ]:

print(f"Starting Test on {len(observations)} sequences...")

num_samples_to_test = 5

for i in range(min(num_samples_to_test, len(observations))):


    current_seq = np.array(observations[i])

    test_segment = current_seq[:50]


    prob, beta_matrix = backward_log(test_segment, initial_A, initial_B, initial_pi)


    print(f"\n--- Sequence #{i+1} ---")
    print(f" Length: {len(test_segment)} bases")
    print(f" Likelihood (Probability): {prob}")

    print(f" Beta Matrix Shape: {beta_matrix.shape}")

    print(f" Last row of Beta: {beta_matrix[-1]}")

print("\n Test Complete!")

Starting Test on 4380 sequences...

--- Sequence #1 ---
 Length: 50 bases
 Likelihood (Probability): 73.89824231656475
 Beta Matrix Shape: (50, 2)
 Last row of Beta: [0. 0.]

--- Sequence #2 ---
 Length: 50 bases
 Likelihood (Probability): 74.35925146975252
 Beta Matrix Shape: (50, 2)
 Last row of Beta: [0. 0.]

--- Sequence #3 ---
 Length: 50 bases
 Likelihood (Probability): 75.17780939028066
 Beta Matrix Shape: (50, 2)
 Last row of Beta: [0. 0.]

--- Sequence #4 ---
 Length: 50 bases
 Likelihood (Probability): 75.17780939028066
 Beta Matrix Shape: (50, 2)
 Last row of Beta: [0. 0.]

--- Sequence #5 ---
 Length: 50 bases
 Likelihood (Probability): 72.59033226154563
 Beta Matrix Shape: (50, 2)
 Last row of Beta: [0. 0.]

 Test Complete!


## 6. Gamma Calculation (Expectation part 1)

In [ ]:
def compute_gamma_log_from_alpha_beta(log_alpha, log_beta):
    """
    Computes gamma in log-space, returns gamma + df.
    """

    T, N = log_alpha.shape
    log_gamma = np.zeros((T, N))
    gamma = np.zeros((T, N))
    df_gamma = []

    for t in range(T):
        terms = log_alpha[t] + log_beta[t]

        max_val = np.max(terms)
        log_denom = max_val + np.log(np.sum(np.exp(terms - max_val)))

        log_gamma[t] = terms - log_denom
        gamma[t] = np.exp(log_gamma[t])   # convert for visualization

    df = pd.DataFrame(
        gamma,
        columns=[f"gamma_state_{i}" for i in range(N)],
        index=[t+1 for t in range(T)]
    )

    return log_gamma, df


In [ ]:
import numpy as np
import pandas as pd

# The following functions (forword, backward_algorithm, compute_gamma_from_alpha_beta)
# must be defined elsewhere or imported before this function can run.

def gamma_for_single_sequence(seq, pi, A, B, return_df=True):
    """
    Computes the Gamma matrix (state posterior probabilities) for a single observation sequence.

    Args:
        seq (list/array of ints): The observation sequence (e.g., DNA bases encoded as ints).
        pi (array): Initial State Probability Vector.
        A (NxN matrix): Transition Probability Matrix.
        B (NxM matrix): Emission Probability Matrix.
        return_df (bool): If True, returns the gamma matrix as a pandas DataFrame for visualization.

    Returns:
        gamma (T,N): State probabilities at each time t.
        df_gamma (DataFrame, optional): DataFrame representation of gamma.
        total_prob (float): The likelihood of the observation sequence P(O|lambda).
    """

    # 1. Forward Pass: Calculate Alpha Matrix
    # We ignore the DataFrame output from the forward function here, only taking the alpha matrix.
    alpha, _ = forword(seq, pi, A, B)

    # 2. Backward Pass: Calculate Beta Matrix
    # We also get the total likelihood (total_prob) of the sequence from this step.
    total_prob, beta = backward_log(seq, A, B, pi)

    # 3. Gamma Calculation: Combine Alpha and Beta to get the Posterior Probability
    gamma, df_gamma = compute_gamma_log_from_alpha_beta(alpha, beta)

    # 4. Return Results
    if return_df:
        # If the user requested the visualization DataFrame, return all three results.
        return gamma, df_gamma, total_prob

    # Otherwise, return only the raw gamma matrix and the total likelihood.
    return gamma, total_prob

### Testing

In [ ]:

gamma, df_gamma, total_prob = gamma_for_single_sequence(observations[0], initial_pi,initial_A,initial_B)

# --- 4. OUTPUT RESULTS ---
print("\n--- Results for the first sequence ---")
print(f"Sequence length: {len(observations[0])}")
# Likelihood is usually very small, so we print it in scientific notation
print(f"Total Likelihood (P(O|lambda)): {total_prob:.5e}")
print("\nFirst 10 steps of Gamma (Posterior Probabilities):")
print(df_gamma.head(10))

# Verification of Gamma: The sum of probabilities for each time step must equal 1.
print("\nVerification: Sum of gamma probabilities for each time step (must be 1.0):")
print(df_gamma.sum(axis=1).head(10))

print("\n Test Complete!")


--- Results for the first sequence ---
Sequence length: 207
Total Likelihood (P(O|lambda)): 3.02868e+02

First 10 steps of Gamma (Posterior Probabilities):
    gamma_state_0  gamma_state_1
1        0.724774       0.275226
2        0.295753       0.704247
3        0.543691       0.456309
4        0.498574       0.501426
5        0.506693       0.493307
6        0.504079       0.495921
7        0.511330       0.488670
8        0.471316       0.528684
9        0.465037       0.534963
10       0.542075       0.457925

Verification: Sum of gamma probabilities for each time step (must be 1.0):
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
dtype: float64

 Test Complete!


## 7. Xi Calculation (Expectation part 2)

### **Xi Expectation Calculation (Transition Expectation)**


**1. What is Xi?**

Xi ξ(t, i, j) represents the **probability that the HMM was in state i at time t, and moved to state j at time t+1**, given the full observation sequence and model parameters.

It is defined as:

$$
\xi_t(i,j) = P(q_t = i, q_{t+1} = j \mid O , \lambda)
$$

This value represents the **expected number of transitions** between state i → j.

---

**2. Why Xi is important?**

Xi is the core component used to **update the transition matrix A** inside the Baum-Welch algorithm:

$$
A_{ij} = \frac{\sum_{t=1}^{T-1} \xi_t(i,j)}
{\sum_{t=1}^{T-1} \gamma_t(i)}
$$

So Xi tells the algorithm:

- How many times we *expect* the system to move from state *i* → *j*.
- This is what makes Baum-Welch "learn" the best transition probabilities.

---

**3. Xi Formula (expanded form)**

$$
  \xi_t(i,j) =
  \frac{
  \alpha_t(i)\; A_{ij}\; B_j(O_{t+1})\; \beta_{t+1}(j)
  }{
  \sum_{i'} \sum_{j'} \alpha_t(i') A_{i'j'} B_{j'}(O_{t+1}) \beta_{t+1}(j')}
$$

---

**4. Intuition (Simple Explanation)**



- **αₜ(i)**: probability of reaching state *i* at time t  
- **Aᵢⱼ**: probability of transitioning from *i → j*  
- **Bⱼ(Oₜ₊₁)**: probability that state j emits the next observation  
- **βₜ₊₁(j)**: probability of observing the rest of the sequence from j  

Multiplying them gives the full probability of this transition.

---

**5. Output Shape**

If:
- T = sequence length  
- N = number of hidden states  

Then:

$$
    \xi \in (T-1,\; N,\; N)
$$

A 3D matrix:
- One matrix for each time step t  
- Each matrix is (N × N) transitions i → j

---

**6. Summary**

Xi shows:
  - Expected transitions between every state pair.
  - Used directly to update transition matrix A.
  - Works together with Gamma for EM in Baum-Welch.



###

In [ ]:
def compute_log_xi(seq, log_alpha, log_beta, logA, logB):
    """
    Computes Xi in LOG-SPACE to avoid underflow.
    Xi_t(i,j) = log P(q_t = i, q_{t+1} = j | O)

    Args:
        seq       : observation sequence (list of ints)
        log_alpha : forward log-probabilities (T x N)
        log_beta  : backward log-probabilities (T x N)
        logA      : log transition matrix (N x N)
        logB      : log emission matrix (N x M)

    Returns:
        log_xi    : Xi in log-space (T-1, N, N)
        df_list   : list of DataFrames of exp(log_xi) for visualization
    """

    T = len(seq)
    N = logA.shape[0]

    log_xi = np.zeros((T - 1, N, N))
    df_list = []

    seq = np.array(seq)

    for t in range(T - 1):

        # ----- Compute denominator using logaddexp over ALL transitions -----
        denom_terms = []
        for i in range(N):
            for j in range(N):
                term = (
                    log_alpha[t, i]
                    + logA[i, j]
                    + logB[j, seq[t+1]]
                    + log_beta[t+1, j]
                )
                denom_terms.append(term)

        # log-sum-exp implemented manually (NO helper)
        max_val = np.max(denom_terms)
        log_denom = max_val + np.log(np.sum(np.exp(denom_terms - max_val)))

        # ----- Compute numerator for each (i, j) -----
        for i in range(N):
            for j in range(N):
                num = (
                    log_alpha[t, i]
                    + logA[i, j]
                    + logB[j, seq[t+1]]
                    + log_beta[t+1, j]
                )
                log_xi[t, i, j] = num - log_denom   # log division

        # ----- Visualization only (convert back to normal probabilities) -----
        df_list.append(
            pd.DataFrame(
                np.exp(log_xi[t]),  # safe because only for display
                index=[f"from_state_{i}" for i in range(N)],
                columns=[f"to_state_{j}" for j in range(N)]
            )
        )

    return log_xi, df_list


In [ ]:
# Example test
seq = observations[0]

alpha, _ = forword(seq, initial_pi, initial_A, initial_B)
total_prob, beta = backward_log(seq, initial_A, initial_B, initial_pi)

xi, df_xi_list = compute_log_xi(seq, alpha, beta,initial_A, initial_B)

print("Xi shape:", xi.shape)
print("\nXi at time t=1:")
df_xi_list[0]


Xi shape: (206, 2, 2)

Xi at time t=1:


,to_state_0,to_state_1
from_state_0,0.183503,0.541270
from_state_1,0.112250,0.162977


## 8. Maximization update



In [ ]:
def update_hmm_parameters_log(log_gamma, log_xi, seq, N, M, epsilon=1e-10):
    """
    Update pi, A, B using log-gamma and log-xi.
    """

    gamma = np.exp(log_gamma)   # normal prob
    xi = np.exp(log_xi)

    T = len(seq)

    # ---- Update pi ----
    pi_new = gamma[0]

    # ---- Update A ----
    A_new = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            numerator = np.sum(xi[:, i, j])
            denominator = np.sum(gamma[:-1, i]) + epsilon
            A_new[i, j] = numerator / denominator

    # ---- Update B ----
    B_new = np.zeros((N, M))
    seq = np.array(seq)
    for j in range(N):
        for k in range(M):
            mask = (seq == k).astype(float)
            numerator = np.sum(gamma[:, j] * mask)
            denominator = np.sum(gamma[:, j]) + epsilon
            B_new[j, k] = numerator / denominator

    return pi_new, A_new, B_new


## 9. Baum Welch Training Loop

###Baum-Welch loop with iterations

In [ ]:
def baum_welch_log(obs, log_pi, log_A, log_B,
                   max_iter=50, tol=1e-5):
    """
    Full Baum-Welch Training Loop in Log-Space.
    -------------------------------------------
    obs       : encoded observation sequence
    log_pi    : log(initial probabilities)
    log_A     : log(transition matrix)
    log_B     : log(emission matrix)
    max_iter  : maximum training iterations
    tol       : convergence threshold
    """

    old_log_likelihood = -np.inf
    N = log_A.shape[0]   # number of states
    M = log_B.shape[1]   # number of observation symbols

    for iteration in range(max_iter):

        # ----------------------------- #
        # 1. Forward (log-space)
        # ----------------------------- #
        log_alpha, _ = forword(obs, log_pi, log_A, log_B)

        # ----------------------------- #
        # 2. Backward (log-space)
        # ----------------------------- #
        log_likelihood, log_beta = backward_log(obs, log_A, log_B, log_pi)

        # ----------------------------- #
        # Stopping Condition
        # ----------------------------- #
        if abs(log_likelihood - old_log_likelihood) < tol:
            print(f"Converged after {iteration+1} iterations.")
            break

        old_log_likelihood = log_likelihood

        # ----------------------------- #
        # 3. Compute gamma (log-space)
        # ----------------------------- #
        gamma, _ = compute_gamma_log_from_alpha_beta(log_alpha, log_beta)

        # ----------------------------- #
        # 4. Compute xi (log-space)
        # ----------------------------- #
        xi, _ = compute_log_xi(obs, log_alpha, log_beta, log_A, log_B)

        # ----------------------------- #
        # 5. Update parameters
        # (returns normal-space values)
        # ----------------------------- #
        pi_new, A_new, B_new = update_hmm_parameters_log(gamma, xi, obs, N, M)

        # ----------------------------- #
        # 6. Convert back to log-space
        # ----------------------------- #
        log_pi = np.log(pi_new)
        log_A  = np.log(A_new)
        log_B  = np.log(B_new)

        print(f"Iteration {iteration+1} completed. "
              f"Log-likelihood = {log_likelihood:.4f}")

    return log_pi, log_A, log_B, log_likelihood


In [ ]:
# Convert initial parameters to log-space
log_pi = np.log(initial_pi)
log_A  = np.log(initial_A)
log_B  = np.log(initial_B)

log_pi, log_A, log_B, final_LL = baum_welch_log(
    observations[0],
    log_pi,
    log_A,
    log_B,
    max_iter=50,
    tol=1e-5
)

print("\nFinal trained parameters:")
print("pi:\n", np.exp(log_pi))
print("\nA:\n", np.exp(log_A))
print("\nB:\n", np.exp(log_B))
print("\nFinal log-likelihood:", final_LL)


Iteration 1 completed. Log-likelihood = -324.1629
Iteration 2 completed. Log-likelihood = -256.4244
Iteration 3 completed. Log-likelihood = -256.2835
Iteration 4 completed. Log-likelihood = -256.1839
Iteration 5 completed. Log-likelihood = -256.1105
Iteration 6 completed. Log-likelihood = -256.0545
Iteration 7 completed. Log-likelihood = -256.0105
Iteration 8 completed. Log-likelihood = -255.9754
Iteration 9 completed. Log-likelihood = -255.9467
Iteration 10 completed. Log-likelihood = -255.9230
Iteration 11 completed. Log-likelihood = -255.9031
Iteration 12 completed. Log-likelihood = -255.8862
Iteration 13 completed. Log-likelihood = -255.8716
Iteration 14 completed. Log-likelihood = -255.8590
Iteration 15 completed. Log-likelihood = -255.8480
Iteration 16 completed. Log-likelihood = -255.8382
Iteration 17 completed. Log-likelihood = -255.8295
Iteration 18 completed. Log-likelihood = -255.8217
Iteration 19 completed. Log-likelihood = -255.8146
Iteration 20 completed. Log-likelihood =

## 10. Viterbi Decoding

In [ ]:
import numpy as np
# Assuming N, M, and the trained parameters pi, A, B are available globally or passed to the function

def viterbi_decoding(seq, pi, A, B):


    T = len(seq)
    N = A.shape[0]
    delta = np.zeros((T, N))

    psi = np.zeros((T, N), dtype=int)

    for i in range(N):
        delta[0, i] = np.log(pi[i]) + np.log(B[i, seq[0]])

    for t in range(1, T):
        for j in range(N):

            log_probs = delta[t-1, :] + np.log(A[:, j])

            max_log_prob = np.max(log_probs)

            delta[t, j] = max_log_prob + np.log(B[j, seq[t]])

            psi[t, j] = np.argmax(log_probs)

    best_prob_log = np.max(delta[T-1, :])
    best_prob = np.exp(best_prob_log)

    Q_T_star = np.argmax(delta[T-1, :])

    best_path = np.zeros(T, dtype=int)
    best_path[T-1] = Q_T_star

    for t in range(T-2, -1, -1):
        best_path[t] = psi[t+1, best_path[t+1]]

    return best_path, best_prob

In [ ]:

new_dna_sequence_str = "CAGTTACGGACG"
new_dna_sequence = np.array([mapping[ch] for ch in new_dna_sequence_str])

viterbi_path, viterbi_prob = viterbi_decoding(new_dna_sequence, final_pi, final_A, final_B)

print("\n--- Viterbi Decoding Results ---")
print(f"DNA Sequence:    {new_dna_sequence_str}")
print(f"Encoded Sequence: {new_dna_sequence}")
print(f"Most Probable Path: {viterbi_path}")
print(f"Path Likelihood: {viterbi_prob:.4e}")

mapping_states = {0: "N", 1: "C"}

decoded_path_str = "".join([mapping_states[state] for state in viterbi_path])
print(f"Decoded States:   {decoded_path_str}")


--- Viterbi Decoding Results ---
DNA Sequence:    CAGTTACGGACG
Encoded Sequence: [1 0 2 3 3 0 1 2 2 0 1 2]
Most Probable Path: [0 0 0 1 1 0 0 0 0 0 0 0]
Path Likelihood: 2.5583e-17
Decoded States:   NNNCCNNNNNNN
